<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/w2d3_t2_videos/tutorials/W2D3_ModernRecurrentNeuralNetworks/W2D3_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuromatch Academy: Week XX, Day YY, Tutorial ZZ  
# Tutorial place holder

__Content creators:__ Name Sirname, Name Sirname

__Content reviewers:__ Name Sirname, Name Sirname

__Content editors:__ Name Sirname, Name Sirname

In [1]:
import numpy as np  